In [1]:
import os
import sys

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sys.path.insert(0, '../src')

from utils import load_data
from features import *

In [3]:
%%time
mbit_rate = 1/125000
data_folder = '../data/'

df_240_lst = load_data('240p', data_folder)
df_360_lst = load_data('360p', data_folder)
df_480_lst = load_data('480p', data_folder)
df_720_lst = load_data('720p', data_folder)
df_1080_lst = load_data('1080p', data_folder)

chunk_size = 300
chunk_240_lst = sum([chunk_data(df, chunk_size) for df in df_240_lst], [])
chunk_360_lst = sum([chunk_data(df, chunk_size) for df in df_360_lst], [])
chunk_480_lst = sum([chunk_data(df, chunk_size) for df in df_480_lst], [])
chunk_720_lst = sum([chunk_data(df, chunk_size) for df in df_720_lst], [])
chunk_1080_lst = sum([chunk_data(df, chunk_size) for df in df_1080_lst], [])

Wall time: 9.97 s


## Aggregate Features
In our EDA, we saw significant differences in aggregate statistics such as the mean and standard deviation. We reconfirm this by taking our chunked data and performing said operations. There seems to be alot of potential colinearity between the bytes and packet stream statistics (strong positive correlation). In our model, we chose to take primarily aggregate features of just the download stream of bytes as a way of thresholding bandwidth usage. Another feature we tried was the difference in download and upload bytes. 

Interestingly, it seems that there are sub-groups within our data as well. For many of the aggregate statistics, we see that 360p and 480p have similar values as well as 720p and 1080p. This is sub-grouping can be seen if we reference back to our EDA plots. These 2 pairs of resolutions have a similar download byte stream graph within the group.

In [4]:
def mean_features_all(df_lst):
  mean_feat = [[
    np.mean(df['2->1Bytes']) * mbit_rate,
    np.std(df['2->1Bytes']) * mbit_rate,
    np.mean(df['1->2Bytes']) * mbit_rate,
    np.std(df['1->2Bytes']) * mbit_rate,
    np.mean(df['2->1Pkts'] - df['1->2Pkts'])  
  ] for df in df_lst]
  return np.array(mean_feat)

In [5]:
agg_240 = mean_features_all(chunk_240_lst)
np.mean(agg_240, axis=0), np.round(np.ptp(agg_240, axis=0), 3), np.round(np.std(agg_240, axis=0), 3)

(array([ 0.33998753,  1.29085521,  0.02461379,  0.07545916, 12.19973584]),
 array([ 0.739,  2.58 ,  0.066,  0.251, 29.037]),
 array([0.193, 0.496, 0.014, 0.038, 7.391]))

In [6]:
agg_360 = mean_features_all(chunk_360_lst)
np.mean(agg_360, axis=0), np.round(np.ptp(agg_360, axis=0), 3), np.round(np.std(agg_360, axis=0), 3)

(array([ 0.50520972,  2.00816564,  0.03052827,  0.10444718, 22.07515941]),
 array([1.0220e+00, 3.8510e+00, 5.9000e-02, 1.9100e-01, 6.5113e+01]),
 array([ 0.291,  0.834,  0.016,  0.047, 14.658]))

In [7]:
agg_480 = mean_features_all(chunk_480_lst)
np.mean(agg_480, axis=0), np.round(np.ptp(agg_480, axis=0), 3), np.round(np.std(agg_480, axis=0), 3)

(array([ 0.78550155,  2.68893204,  0.04683803,  0.14326982, 34.84812495]),
 array([ 1.423,  2.689,  0.103,  0.291, 86.197]),
 array([ 0.368,  0.719,  0.023,  0.058, 20.81 ]))

In [8]:
agg_720 = mean_features_all(chunk_720_lst)
np.mean(agg_720, axis=0), np.round(np.ptp(agg_720, axis=0), 3), np.round(np.std(agg_720, axis=0), 3)

(array([ 1.52541254,  3.68402735,  0.0836291 ,  0.19199595, 69.37795847]),
 array([  3.137,   4.012,   0.205,   0.328, 181.11 ]),
 array([ 0.906,  1.027,  0.051,  0.077, 44.622]))

In [9]:
agg_1080 = mean_features_all(chunk_1080_lst)
np.mean(agg_1080, axis=0), np.round(np.ptp(agg_1080, axis=0), 3), np.round(np.std(agg_1080, axis=0), 3)

(array([  2.10579994,   4.38608986,   0.11426992,   0.2439531 ,
        100.17294031]),
 array([  4.341,   5.44 ,   0.539,   1.572, 251.642]),
 array([ 1.158,  1.17 ,  0.081,  0.179, 61.968]))

## Peak Related Aggregate Features
Peaks were a strong point of focus in our EDA. The term "peak" is a bit of a misnomer - a "peak" is simply a large data transaction between 2 IP addresses at a single second (observation). "Large" is relative to the scale and intensity of internet activity. We experimented with both a relative threshold and a hard threshold to define was a peak. We found that using a relative threshold did well to preserve much of the data behavior but it did not show us clear distinction between lower resolutions (240p, 360p, 480p). We were wary of using a hard threshold as network conditions can vary from user to user but using a predefined lower boundary helped show a more a difference between resolutions.

Much like our basic aggregate features, we perform similar calculations to characterize the peaks for each resolution. We also begin to delve a bit into the frequency domain (though not really). Intuitively, we should expect more data spikes as the resolution increases and the raw number itself can be useful feature. However, we are more interested in the time between these spikes and the ratio of seconds:spikes. 

In [10]:
%%time
mbps = 10
peaks_240 = [peak_features(df, '2->1Bytes', mbps) for df in chunk_240_lst]
peaks_360 = [peak_features(df, '2->1Bytes', mbps) for df in chunk_360_lst]
peaks_480 = [peak_features(df, '2->1Bytes', mbps) for df in chunk_480_lst]
peaks_720 = [peak_features(df, '2->1Bytes', mbps) for df in chunk_720_lst]
peaks_1080 = [peak_features(df, '2->1Bytes', mbps) for df in chunk_1080_lst]

Wall time: 5.15 s


In [11]:
np.mean(peaks_240, axis=0)

array([  4.9102591 ,   0.42575192,   0.46153846, 114.1025641 ])

In [12]:
np.mean(peaks_360, axis=0)

array([11.40987868,  1.80985837,  4.11688312, 48.63574521])

In [13]:
np.mean(peaks_480, axis=0)

array([13.8871883 ,  1.8124035 ,  7.80769231, 22.271445  ])

In [14]:
np.mean(peaks_720, axis=0)

array([13.93255861,  1.99586214, 16.15584416,  9.74875943])

In [15]:
np.mean(peaks_1080, axis=0)

array([14.07435176,  2.60823288, 25.48717949,  6.75123694])

## Spectral Features

We know from our initial EDA that, generally, higher resolution data is high frequency and lower resolution data is low frequency. There are the corner cases where, in a high-res video, there is just a simple image or some text on a background. The resultant byte stream graph would show a very sparse amount of spikes (in relation to what we would normally expect from high-res) and taking the Fourier Transform/Welch's method would return a periodogram that suggests the strongest signal is a low frequency. 

We attempt to combat these corner cases by taking the magnitude (or power) of the signal into consideration as well. Just like in our aggregate bytes, we see that the magnitude of power in high-res data is significantly larger than its low-res counterpart. We take both the maximum power in the periodogram as well as calculate its peak prominence. We suspect that this peak prominence feature will help create thresholds fo te resolution as low-res periodograms are much noiser than high-res.

We settled on resampling/binning our current data into 2 second samples. Although we have the ability to take millisecond samples to capture more high frequency signals, we saw that the high frequency data tends to hover the .2 -.3Hz range. Nyquist's theorem states that we are able to capture frequencies that are half of our sampling rate. So the 2 second bins -> .5 sample/second gives us the most information as we are primarily interested in the 0 - .3Hz frequency range.

For the actual frequency values, we took the area under the curve of the periodogram for bins on the frequency. We are trying to capture the peaks in the periodogram but looking at only the frequencies with the highest power typically does not give us an accurate representation. Taking the area under the curve of bins, however, ensures that we know whether there is a stronger signal to be seen in a certain range.


In [16]:
%%time
size='1000ms'
resample_240_1s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_240_lst]
resample_360_1s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_360_lst]
resample_480_1s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_480_lst]
resample_720_1s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_720_lst]
resample_1080_1s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_1080_lst]

Wall time: 33.6 s


In [17]:
%%time
size='2000ms'
resample_240_2s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_240_lst]
resample_360_2s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_360_lst]
resample_480_2s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_480_lst]
resample_720_2s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_720_lst]
resample_1080_2s = [explode_extended(df).resample(size, on='Time').sum() for df in chunk_1080_lst]

Wall time: 34.9 s


In [19]:
## basic spectral features - locating frequency of strongest signal; more thresholding
sampling_rate = .5

prominence_240 = [spectral_features(df, 'pkt_size', sampling_rate) for df in resample_240_1s]
prominence_360 = [spectral_features(df, 'pkt_size', sampling_rate) for df in resample_360_1s]
prominence_480 = [spectral_features(df, 'pkt_size', sampling_rate) for df in resample_480_1s]
prominence_720 = [spectral_features(df, 'pkt_size', sampling_rate) for df in resample_720_1s]
prominence_1080 = [spectral_features(df, 'pkt_size', sampling_rate) for df in resample_1080_1s]

In [20]:
np.mean(prominence_240, axis=0)

array([1.46244025, 5.53102656])

In [21]:
np.mean(prominence_360, axis=0)

array([2.12352032, 8.10010269])

In [22]:
np.mean(prominence_480, axis=0)

array([ 3.15187929, 11.81268701])

In [23]:
np.mean(prominence_720, axis=0)

array([ 4.63248343, 18.97451135])

In [24]:
np.mean(prominence_1080, axis=0)

array([ 6.02828232, 27.5141869 ])

In [ ]:
## area under the curve for a periodogram

In [26]:
bins = 3
pxx_den_240 = [power_density(df, bins) for df in resample_240_2s]
pxx_den_360 = [power_density(df, bins) for df in resample_360_2s]
pxx_den_480 = [power_density(df, bins) for df in resample_480_2s]
pxx_den_720 = [power_density(df, bins) for df in resample_720_2s]
pxx_den_1080 = [power_density(df, bins) for df in resample_1080_2s]

pxx_den_df = pd.DataFrame({
  "240p": np.mean(pxx_den_240, axis=0),
  "360p": np.mean(pxx_den_360, axis=0),
  "480p": np.mean(pxx_den_480, axis=0),
  "720p": np.mean(pxx_den_720, axis=0),
  "1080p": np.mean(pxx_den_1080, axis=0)
}).T

#pxx_den_df.columns = ['[-.001, .086)', '[.086, .172)', '[.172, .259)']
pxx_den_df['St_dev'] = [
  np.mean(np.std(pxx_den_240, axis=1)),
  np.mean(np.std(pxx_den_360, axis=1)),
  np.mean(np.std(pxx_den_480, axis=1)),
  np.mean(np.std(pxx_den_720, axis=1)), 
  np.mean(np.std(pxx_den_1080, axis=1))]

pxx_den_df

,0,1,St_dev
240p,0.485731,0.499517,0.029179
360p,0.498427,0.487698,0.025104
480p,0.499509,0.487133,0.030423
720p,0.474390,0.510991,0.035803
1080p,0.442594,0.543255,0.080933


In [28]:
bins = 4
pxx_den_240 = [power_density(df, bins) for df in resample_240_2s]
pxx_den_360 = [power_density(df, bins) for df in resample_360_2s]
pxx_den_480 = [power_density(df, bins) for df in resample_480_2s]
pxx_den_720 = [power_density(df, bins) for df in resample_720_2s]
pxx_den_1080 = [power_density(df, bins) for df in resample_1080_2s]

pxx_den_df = pd.DataFrame({
  "240p": np.mean(pxx_den_240, axis=0),
  "360p": np.mean(pxx_den_360, axis=0),
  "480p": np.mean(pxx_den_480, axis=0),
  "720p": np.mean(pxx_den_720, axis=0),
  "1080p": np.mean(pxx_den_1080, axis=0)
}).T

#pxx_den_df.columns = ['[-.001, .086)', '[.086, .172)', '[.172, .259)']
pxx_den_df['St_dev'] = [
  np.mean(np.std(pxx_den_240, axis=1)),
  np.mean(np.std(pxx_den_360, axis=1)),
  np.mean(np.std(pxx_den_480, axis=1)),
  np.mean(np.std(pxx_den_720, axis=1)), 
  np.mean(np.std(pxx_den_1080, axis=1))]

pxx_den_df

,0,1,2,St_dev
240p,0.307911,0.353852,0.311066,0.031773
360p,0.315678,0.351877,0.302144,0.033028
480p,0.312770,0.358190,0.303528,0.036072
720p,0.260273,0.378884,0.334207,0.061714
1080p,0.236666,0.368455,0.369140,0.088676


## Rolling Windows

In [59]:
window = 10
normalized_std_240 = [rolling_normalized_std(df, window) for df in resample_240_1s]
normalized_std_360 = [rolling_normalized_std(df, window) for df in resample_360_1s]
normalized_std_480 = [rolling_normalized_std(df, window) for df in resample_480_1s]
normalized_std_720 = [rolling_normalized_std(df, window) for df in resample_720_1s]
normalized_std_1080 = [rolling_normalized_std(df, window) for df in resample_1080_1s]

CoV on a rolling window average

In [60]:
np.mean(normalized_std_240, axis=0)

1.2089628807853683

In [61]:
np.mean(normalized_std_360, axis=0)

1.3746067802619966

In [62]:
np.mean(normalized_std_480, axis=0)

1.2365629553827493

In [63]:
np.mean(normalized_std_720, axis=0)

0.8249207442529525

In [64]:
np.mean(normalized_std_1080, axis=0)

0.6858985997135151

In [65]:
%%time
feat_lst = ['1->2Bytes', '2->1Bytes']

normalized_std_240 = [normalized_std(df, feat_lst) for df in chunk_240_lst]
normalized_std_360 = [normalized_std(df, feat_lst) for df in chunk_360_lst]
normalized_std_480 = [normalized_std(df, feat_lst) for df in chunk_480_lst]
normalized_std_720 = [normalized_std(df, feat_lst) for df in chunk_720_lst]
normalized_std_1080 = [normalized_std(df, feat_lst) for df in chunk_1080_lst]

Wall time: 1.48 s


cofficient of variation - download & upload

In [66]:
np.mean(normalized_std_240, axis=0)

array([3.36892964, 4.43068193])

In [67]:
np.mean(normalized_std_360, axis=0)

array([3.76490266, 4.65959127])

In [68]:
np.mean(normalized_std_480, axis=0)

array([3.27156646, 3.89949206])

In [69]:
np.mean(normalized_std_720, axis=0)

array([2.63876626, 2.95102722])

In [70]:
np.mean(normalized_std_1080, axis=0)

array([2.45984453, 2.59694815])